# **UNIVERSIDAD NACIONAL DE SAN ANTONIO ABAD DEL CUSCO**
# **DEPARTAMENTO ACADÉMICO DE INFORMÁTICA**
## **DEEP LEARNING**




# **PRÁCTICA Nº 03**
## **GRADIENTES Y DIFERENCIACIÓN AUTOMÁTICA**

**1.   OBJETIVOS**


*   Conocer la diferenciación automática para calcular la derivada de una función
*  Aplicar la diferenciación automática al entrenamiento de redes neuronales

### **1.0 — Introducción**

tf. GradientTape nos permite realizar un seguimiento de los cálculos de TensorFlow y calcular gradientes (con respecto a) algunas variables dadas.
Por ejemplo, podríamos realizar un seguimiento de los siguientes cálculos y calcular gradientes de la siguiente manera:tf.GradientTape

In [1]:
import tensorflow as tf
print(tf.__version__) 

2.12.0


In [ ]:
x = tf.constant(5.0)
with tf.GradientTape() as tape:
    tape.watch(x)
    y = x**3

print(tape.gradient(y, x).numpy())

del tape, x

75.0


- Por defecto, no realiza un seguimiento de las constantes, por lo que 
debemos indicarle que: GradientTapetape.watch(variable)
- Luego podemos realizar algún cálculo sobre las variables que estamos 
observando. El cálculo puede ser cualquier cosa, desde cubing, , hasta pasarlo a través de una red neuronal.x**3
- Calculamos gradientes de un cálculo con una variabl. 

Nota, devuelve un archivo que puede convertir a formato ndarray con tape.gradient(target, sources)tape.gradientEagerTensor.numpy()

Si en algún momento, queremos usar múltiples variables en nuestros cálculos, todo lo que tenemos que hacer es dar una lista o tupla de esas variables. Cuando optimizamos los modelos Keras, pasamos como nuestra lista de variables.tape.gradientmodel.trainable_variables

### **1.1 — Observación automática de variables**

Si fuera una variable entrenable en lugar de una constante, no habría necesidad de decirle a la cinta que la vea: observa automáticamente todas las variables entrenables.xGradientTape

In [ ]:
x = tf.Variable(6.0, trainable=True) # Error on: tf.constant(6.0) or tf.Variable(6.0, trainable=False)
with tf.GradientTape() as tape:
    y = x**3

print(tape.gradient(y, x).numpy())

del tape, x

108.0


### **1.2 — watch_accessed_variables=Falso**
Si no queremos ver todas las variables entrenables automáticamente, podemos establecer el parámetro de la cinta en: GradientTape(watch_accessed_variables=False)



In [ ]:

x = tf.Variable(3.0, trainable=True)
with tf.GradientTape(watch_accessed_variables=False) as tape:
    y = x**3

print(tape.gradient(y, x))

del tape, x

None


Deshabilitar nos da un buen control sobre qué variables queremos ver con watch_accessed_variables

Si tiene muchas variables entrenables y no las está optimizando todas a la vez, es posible que desee deshabilitarlas para protegerse de errores.

### **1.3 — Derivadas de orden superior**

Si desea calcular derivadas de orden superior, puede utilizar anidados:GradientTapes

In [ ]:

x = tf.Variable(3.0, trainable=True)
with tf.GradientTape() as tape1:
    with tf.GradientTape() as tape2:
        y = x ** 3
    order_1 = tape2.gradient(y, x)
order_2 = tape1.gradient(order_1, x)

print(order_2.numpy())

del x, tape1, tape2, order_1, order_2

18.0


Las derivadas de orden superior son generalmente el único momento en que desea calcular gradientes dentro de un objeto. De lo contrario, ralentizará los cálculos a medida que observa cada cálculo realizado en el gradiente.GradientTapeGradientTape



### **1.4 — persistente=True**

Si tuviéramos que ejecutar lo siguiente:

In [ ]:

a = tf.Variable(6.0, trainable=True)
b = tf.Variable(2.0, trainable=True)
with tf.GradientTape(persistent=True) as tape:
    y1 = a ** 2
    y2 = b ** 3
                                                                                                                                                                                                                                                                                                                                                
print(tape.gradient(y1, a).numpy())
print(tape.gradient(y2, b).numpy())

del a, b, tape, y1, y2

12.0
12.0


Pero en realidad, llamar por segunda vez generará un error.tape.gradient

Esto se debe a que inmediatamente después de llamar, libera toda la información almacenada en su interior con fines computacionales.tape.gradientGradientTape

Si queremos evitar esto, podemos configurar persistent=True

### **1.5 — stop_recording()**

tape.stop_recording() pausa temporalmente la grabación de las cintas, lo que lleva a una mayor velocidad de cálculo.

In [ ]:
x = tf.Variable(3.0, trainable=True)
with tf.GradientTape() as tape:
    y = x**3
    with tape.stop_recording():
        # Poner tape.gradient fuera de un bloque stop_recording, pero dentro del bloque de cinta, generará una advertencia sobre la ineficiencia.
        print(tape.gradient(y, x).numpy())

del tape, x

27.0


En funciones largas, es más legible usar bloques varias veces para calcular gradientes en medio de una función, que calcular todos los gradientes al final de una función.stop_recording

In [ ]:
a = tf.Variable(6.0, trainable=True)
b = tf.Variable(2.0, trainable=True)
with tf.GradientTape(persistent=True) as tape:
    y1 = a ** 2
    with tape.stop_recording():
        print(tape.gradient(y1, a).numpy())
    
    y2 = b ** 3
    with tape.stop_recording():
        print(tape.gradient(y2, b).numpy())
                                                                                                                                                                                                                                                                                                                                                

del a, b, tape, y1, y2

12.0
12.0


El efecto es menos notable y posiblemente incluso lo contrario para un pequeño ejemplo, sin embargo, para una gran parte del código, creo que los bloques ayudan con creces a mejorar la legibilidad.stop_recording

### **1.6 — Otros métodos**

Aunque no se entrará en detalles aquí, tiene algunos otros métodos útiles, que incluyen:GradientTape

- jacobian: "Calcula el jacobiano usando operaciones grabadas en el contexto de esta cinta."
- batch_jacobian: "Calcula y apila por ejemplo jacobianos".
- reset: "Borra toda la información almacenada en esta cinta".
- watched_variables: "Devuelve las variables observadas por esta cinta en orden de construcción".
Toda la información anterior citada de la documentación de GradientTape.

## **Usos avanzados**

### **2.0 — Regresión lineal**

Para comenzar con los usos más avanzados de GradientTape, veamos un clásico "¡Hola mundo!" a ML: regresión lineal.

Primero, comenzamos definiendo algunas variables y funciones esenciales.

In [1]:
import random
import numpy as np
import tensorflow as tf
#------- programa principal--------------

puntos = np.genfromtxt("/content/data - data.csv", delimiter=",")
x = []
# Función de pérdida
def loss(real_y, pred_y):
    return tf.abs(real_y - pred_y)

# Datos de entrenamiento
x_train = x
y_train = np.asarray([(i*6.12)+1.35 for i in x_train]) # y = 6.12x+1.35

# Variables entrenables
a = tf.Variable(random.random(), trainable=True)
b = tf.Variable(random.random(), trainable=True)

# función de paso
def step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # Hacer predicción
        pred_y = a * real_x + b
        # Cáclulo de pérdida
        reg_loss = loss(real_y, pred_y)
    
    # Cacula gradientes
    a_gradients, b_gradients = tape.gradient(reg_loss, (a, b))

    # actualiza las variables
    a.assign_sub(a_gradients * 0.001)
    b.assign_sub(b_gradients * 0.001)

# ciclo de entrenamiento
for _ in range(10000):
    step(x_train, y_train)

print(f'y ≈ {a.numpy()}x + {b.numpy()}')

del a, b, x_train, y_train, step, loss

y ≈ 0.027295438572764397x + 0.500281572341919


Luego, podemos seguir adelante y definir nuestra función de paso. La función de paso se ejecutará en cada época para actualizar las variables entrenables, a y b



### **2.1 — Regresión polinómica**

Podemos ampliar rápidamente el ejemplo anterior para trabajar con cualquier polinomio.

Simplemente cambie las variables que estamos usando y la ecuación que estamos optimizando.

In [ ]:

import random
import numpy as np

# función de pérdida
def loss(real_y, pred_y):
    return tf.abs(real_y - pred_y)

# Datos de entrenamiento
x_train = np.asarray([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y_train = np.asarray([6*i**2 + 8*i + 2 for i in x_train]) # y = 6x^2 + 8x + 2

# variables entrenables
a = tf.Variable(random.random(), trainable=True)
b = tf.Variable(random.random(), trainable=True)
c = tf.Variable(random.random(), trainable=True)

# función de paso
def step(real_x, real_y):
    with tf.GradientTape(persistent=True) as tape:
        # hacer predicción
        pred_y = a*real_x**2 + b*real_x + c
        # Calcular pérdida
        poly_loss = loss(real_y, pred_y)
    
    # Calcular gradientes
    a_gradients, b_gradients, c_gradients = tape.gradient(poly_loss, (a, b, c))

    # actualizar variables
    a.assign_sub(a_gradients * 0.001)
    b.assign_sub(b_gradients * 0.001)
    c.assign_sub(c_gradients * 0.001)

# Ciclo de entrenamiento
for _ in range(10000):
    step(x_train, y_train)

print(f'y ≈ {a.numpy()}x^2 + {b.numpy()}x + {c.numpy()}')

del a, b, x_train, y_train, step, loss

#y = 6.002356052398682x^2 + 7.548577308654785x + 1.9996548891067505



y ≈ 5.961971282958984x^2 + 7.590671539306641x + 1.999351143836975


### **2.2 — Clasificación de MNIST**

La regresión polinómica es divertida y todo, pero el verdadero problema es optimizar las redes neuronales.

Afortunadamente para nosotros, poco tiene que cambiar de los ejemplos anteriores para hacer precisamente eso.

Comenzamos siguiendo el procedimiento estándar, cargando los datos, preprocesándolos y configurando los hiperparámetros.

In [ ]:
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
import matplotlib.pyplot as plt
import random
import math
%matplotlib inline

In [ ]:
# Load and pre-process training data
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = (x_train / 255)
x_test = (x_test / 255)


In [ ]:
# Hyperparameters
batch_size = 128
epochs = 25
optimizer = SGD(lr=0.001)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
model = Sequential()
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(10))



In [ ]:
def step(real_x, real_y):
  with tf.GradientTape() as tape: # Record the gradient calculation
    # Flatten x, [b, 28, 28] => [b, 784]
    x = tf.reshape(x_train, (-1, 28*28))
    # Step1. get output [b, 784] => [b, 10]
    out = model(x)
    #  [b] => [b, 10]
    y_onehot = tf.one_hot(y_train, depth=10)
    # Calculate squared error, [b, 10]
    loss = tf.square(out-y_onehot)
    # Calculate the mean squared error, [b]
    loss = tf.reduce_sum(loss) / x.shape[0]
    print('pérdida:',loss)
  #Step3. Calculate gradients w1, w2, w3, b1, b2, b3
  grads = tape.gradient(loss, model.trainable_variables)
  # Auto gradient calculation
  #grads = tape.gradient(loss, model.trainable_variables)
  # w' = w - lr * grad, update parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

In [ ]:
# Training loop
bat_per_epoch = math.floor(len(x_train) / batch_size)
for epoch in range(epochs):
    print('=', end='')
    for i in range(bat_per_epoch):
      step(x_train[i], y_train[i])

=pérdida: tf.Tensor(2.1264305, shape=(), dtype=float32)
pérdida: tf.Tensor(2.030654, shape=(), dtype=float32)
pérdida: tf.Tensor(1.9466203, shape=(), dtype=float32)
pérdida: tf.Tensor(1.872545, shape=(), dtype=float32)
pérdida: tf.Tensor(1.8069487, shape=(), dtype=float32)
pérdida: tf.Tensor(1.7486014, shape=(), dtype=float32)
pérdida: tf.Tensor(1.6965092, shape=(), dtype=float32)
pérdida: tf.Tensor(1.6498291, shape=(), dtype=float32)
pérdida: tf.Tensor(1.6078732, shape=(), dtype=float32)
pérdida: tf.Tensor(1.5700306, shape=(), dtype=float32)
pérdida: tf.Tensor(1.5357925, shape=(), dtype=float32)
pérdida: tf.Tensor(1.5047246, shape=(), dtype=float32)
pérdida: tf.Tensor(1.476453, shape=(), dtype=float32)
pérdida: tf.Tensor(1.4506584, shape=(), dtype=float32)
pérdida: tf.Tensor(1.4270588, shape=(), dtype=float32)
pérdida: tf.Tensor(1.4054135, shape=(), dtype=float32)
pérdida: tf.Tensor(1.3855071, shape=(), dtype=float32)
pérdida: tf.Tensor(1.3671602, shape=(), dtype=float32)
pérdida: tf.

KeyboardInterrupt: ignored

### **TAREA**

Utilizar la diferenciación automática para calcular los parámetros de la regresión lineal aplicada a los datos proporcionados y comparar con los resultados de la práctica 02.